In [25]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_parquet("hf://datasets/Paper2Poster/Paper2Poster/data/train-00000-of-00001.parquet")

In [2]:
df.head()

,title,conference,year,paper_url,image_url,qa
0,Biologically-inspired adaptive learning in the...,NeurIPS,2023,https://openreview.net/pdf?id=WjHYgEfXiV,https://neurips.cc/media/PosterPDFs/NeurIPS%20...,"{\n ""detail"": {\n ""questions"": {\n ..."
1,The Hessian Screening Rule,NeurIPS,2022,https://proceedings.neurips.cc/paper_files/pap...,https://neurips.cc/media/PosterPDFs/NeurIPS%20...,"{\n ""detail"": {\n ""questions"": {\n ..."
2,"Transformers, parallel computation, and logari...",ICML,2024,https://arxiv.org/pdf/2402.09268,https://icml.cc/media/PosterPDFs/ICML%202024/3...,"{\n ""detail"": {\n ""questions"": {\n ..."
3,Neural Tangent Kernels for Axis-Aligned Tree E...,ICML,2024,https://arxiv.org/pdf/2109.04983,https://icml.cc/media/PosterPDFs/ICML%202024/3...,"{\n ""detail"": {\n ""questions"": {\n ..."
4,CW-ERM: Improving Autonomous Driving Planning ...,NeurIPS,2022,https://arxiv.org/pdf/2210.02174,https://neurips.cc/media/PosterPDFs/NeurIPS%20...,"{\n ""detail"": {\n ""questions"": {\n ..."


In [2]:
import openreview

# API V2
client = openreview.api.OpenReviewClient(
    baseurl='https://api2.openreview.net',
    username='aaditya.deshpande@tcs.com',
    password='Aadityamd@0207'
)

# API V1
client_v1 = openreview.Client(
    baseurl='https://api.openreview.net',
    username='aaditya.deshpande@tcs.com',
    password='Aadityamd@0207'
)


In [ ]:
client_v2 = client


In [180]:
venue_id = "NeurIPS.cc/2023/Conference"  # example
venue_group = client_v2.get_group(venue_id)
submission_name = venue_group.content["submission_name"]["value"]
target_titles = {
    "Conditional Matrix Flows for Gaussian Graphical Models",
}

def norm(s):
    return str(s).strip().lower()

def get_title(sub):
    t = sub.content['title']['value'] if isinstance(sub.content.get("title"), dict) else sub.content.get("title")
    #if isinstance(t, dict) and "value" in t:
    #    return t["value"]
    return t

reply_type = "Official_Review"  # or "Meta_Review", "Decision", etc.
submissions = client_v2.get_all_notes(
    invitation=f"{venue_id}/-/{submission_name}",
    details="replies"
)

wanted = {norm(t) for t in target_titles}

replies = []
for submission in submissions:
    title = get_title(submission)
    if norm(title) not in wanted:
        continue

    for reply in submission.details.get("replies", []):
        if any(inv.endswith(reply_type) for inv in reply.get("invitations", [])):
            replies.append({
                "title": title,
                "forum": submission.id,
                "reply": reply
            })

replies


Getting V2 Notes: 100%|█████████▉| 3391/3395 [01:40<00:00, 33.81it/s]


[{'title': 'Conditional Matrix Flows for Gaussian Graphical Models',
  'forum': 'GYnbubCXhE',
  'reply': {'content': {'summary': {'value': 'This paper concerns the estimation of precision matrix under $l_p$ norm sparcity penal. The solution is a variational inference through normalizing flow, which is a function of shrinkage parameter $\\lambda$ and non-negative norm parameter $p$. It allows for straightforward computation of solution paths for the intervals of $\\lambda$ and $p$, and was empirically evaluated on two relatively small data sets.'},
    'soundness': {'value': '3 good'},
    'presentation': {'value': '3 good'},
    'contribution': {'value': '3 good'},
    'strengths': {'value': 'Framework for GGM estimation based on conditional normalizing flows, indeed appears novel. Supporting math seems solid. \n\nUsing simulated annealing algorithm to recover a path of solutions for varying $\\lambda$ and $p$ is useful, in particular for the case of $p$, as in case of $\\lambda$ it wa

In [56]:
type(replies)

list

In [69]:
len(replies)

4

In [58]:
replies[1]

{'title': 'Conditional Matrix Flows for Gaussian Graphical Models',
 'forum': 'GYnbubCXhE',
 'reply': {'content': {'summary': {'value': 'This paper targets the structure learning problem in Gaussian Graphical Models via (Normalising) Flow-based Variational approximation of the elements of weight metrics that correspond to the Gaussian Bayesian network. \nThey use sub-l1 pseudo norms to penalize dense precision metrics (which correspond to graphs with numerous links) without imposing an extra high penalty for large non-zero values (which typically occurs if $l_{\\geq1}$ is used).'},
   'soundness': {'value': '3 good'},
   'presentation': {'value': '4 excellent'},
   'contribution': {'value': '2 fair'},
   'strengths': {'value': '1. Up to my knowledge, this is the first time flows are applied to the space of positive definite matrices. \n2. The proposed approach is flexible meaning the class of applicable prior and likelihood functions is quite large.\n3. Using sub-l1 norm is suitable fo

In [90]:
replies[1].get("reply")

{'content': {'summary': {'value': 'This paper targets the structure learning problem in Gaussian Graphical Models via (Normalising) Flow-based Variational approximation of the elements of weight metrics that correspond to the Gaussian Bayesian network. \nThey use sub-l1 pseudo norms to penalize dense precision metrics (which correspond to graphs with numerous links) without imposing an extra high penalty for large non-zero values (which typically occurs if $l_{\\geq1}$ is used).'},
  'soundness': {'value': '3 good'},
  'presentation': {'value': '4 excellent'},
  'contribution': {'value': '2 fair'},
  'strengths': {'value': '1. Up to my knowledge, this is the first time flows are applied to the space of positive definite matrices. \n2. The proposed approach is flexible meaning the class of applicable prior and likelihood functions is quite large.\n3. Using sub-l1 norm is suitable for structure learning. \n4. The proposed algorithm is mathematically sound (as far as I can follow) and is 

In [ ]:
def norm(s):
    return str(s).strip().lower()

def get_reviews(title, conference, year, reply_type="Official_Review"):
    '''given a title, conference, year, and reply type, fetches summary, strengths, and weaknesses from the each review from OpenReview'''
    venue_id = f"{conference}.cc/{year}/Conference"
    target_title = norm(title)
    reviews = {}
    submissions = client.get_all_notes(invitation=f"{venue_id}/-/{submission_name}", details="replies")
    replies = []
    for submission in submissions:
        t = target_title
        for reply in submission.details.get("replies", []):
            if any(inv.endswith(reply_type) for inv in reply.get("invitations", [])):
                replies.append({"title": t, "forum": submission.id, "reply": reply})
    
    for i in range (len(replies)):
        summary = replies[i].get("reply").get("content").get("summary").get("value") if replies[i].get("reply").get("content").get("summary") else None
        strengths = replies[i].get("reply").get("content").get("strengths").get("value") if replies[i].get("reply").get("content").get("strengths") else None
        weaknesses = replies[i].get("reply").get("content").get("weaknesses").get("value") if replies[i].get("reply").get("content").get("weaknesses") else None
        reviews[f"review_{i+1}"] = {"summary": summary, "strengths": strengths, "weaknesses": weaknesses}
    return reviews


In [71]:
get_reviews("Conditional Matrix Flows for Gaussian Graphical Models", "NeurIPS", 2023, reply_type="Official_Review")

Getting V2 Notes: 100%|█████████▉| 3391/3395 [01:06<00:00, 51.06it/s]


{'review_1': {'summary': 'Paper introduces a framework to detect and categorize different model uncertainty types in classification setting. Proposer model-agnostic (with some assumptions of model structure) uncertainty quantification (UQ) relies on kernel density estimation on latent space representation by defining scores for OOD, Bnd, and IDM examples. Proposed approach is empirically evaluated using toy data as well as real MNIST and UCI datasets. Some of the limitations of existing UQ are examined as well as ability to use new uncertainty categorisation as flagging method to filter out training example to learn more accurate models.',
  'strengths': 'To my knowledge, paper presents a novel approach to categorise different predictive uncertainty types in classification setting. The proposal is well-structured with appropriate theoretical background, and especially practical evaluation which includes both toy illustration of properties as well as real data evaluation of MNIST varian

In [ ]:
def norm(s):
    return str(s).strip().lower()

def get_title(sub):
    t = sub.content.get("title")
    return t.get("value") if isinstance(t, dict) else t

def unwrap(v):
    return v.get("value") if isinstance(v, dict) else v

def get_reviews_fast(title, conference, year, reply_type="Official_Review"):
    venue_id = f"{conference}.cc/{year}/Conference"
    target_title = norm(title)
    reviews = {}

    # --- Try server-side filter by title (fastest) ---
    try:
        submissions = client.get_all_notes(
            invitation=f"{venue_id}/-/{submission_name}",
            content={"title": title},     # server-side filter
            details="replies",
            limit=1
        )
    except Exception:
        submissions = []

    # --- Fallback: scan but break at first match ---
    if not submissions:
        submissions = client_v2.get_all_notes(
            invitation=f"{venue_id}/-/{submission_name}",
            details="replies"
        )

    for submission in submissions:
        sub_title = get_title(submission)
        if norm(sub_title) != target_title:
            # only needed in fallback mode
            continue

        for i, reply in enumerate(submission.details.get("replies", []), start=1):
            if any(inv.endswith(reply_type) for inv in reply.get("invitations", [])):
                content = reply.get("content", {})
                reviews[f"review_{i}"] = {
                    "summary": unwrap(content.get("summary")),
                    "strengths": unwrap(content.get("strengths")),
                    "weaknesses": unwrap(content.get("weaknesses")),
                }

        break  # stop after the matching paper

    return reviews
gg = get_reviews_fast("Conditional Matrix Flows for Gaussian Graphical Models", "NeurIPS", 2023, reply_type="Official_Review")

Getting V2 Notes: 100%|█████████▉| 3391/3395 [01:26<00:00, 39.13it/s]


In [73]:
gg

{'review_8': {'summary': 'This paper concerns the estimation of precision matrix under $l_p$ norm sparcity penal. The solution is a variational inference through normalizing flow, which is a function of shrinkage parameter $\\lambda$ and non-negative norm parameter $p$. It allows for straightforward computation of solution paths for the intervals of $\\lambda$ and $p$, and was empirically evaluated on two relatively small data sets.',
  'strengths': 'Framework for GGM estimation based on conditional normalizing flows, indeed appears novel. Supporting math seems solid. \n\nUsing simulated annealing algorithm to recover a path of solutions for varying $\\lambda$ and $p$ is useful, in particular for the case of $p$, as in case of $\\lambda$ it was fairly straightforward to perform it with other methods too. I am just wondering how costly and scalable it is under the new framework, an empirical/theoretical analysis would be appreciated.',
  'weaknesses': 'Empirical evaluation appears limit

In [74]:
nips_2023_titles

['Biologically-inspired adaptive learning in the Hopfield-network based self-optimization model',
 'Flexible Attention-Based Multi-Policy Fusion for Efficient Deep Reinforcement Learning',
 'Conditional Matrix Flows for Gaussian Graphical Models',
 'Matrix Compression via Randomized Low Rank and Low Precision Factorization',
 'Color Equivariant Convolutional Networks',
 'VideoComposer: Compositional Video Synthesis with Motion Controllability',
 'SiT Dataset: Socially Interactive Pedestrian Trajectory Dataset for Social Navigation Robots',
 'Automatic Clipping: Differentially Private Deep Learning Made Easier and Stronger',
 'A Theoretical Analysis of the Test Error of Finite-Rank Kernel Ridge Regression',
 'Comparing Optimization Targets for Contrast-Consistent Search',
 'Fine-Tuning Language Models with Just Forward Passes',
 'PIXIU: A Comprehensive Benchmark, Instruction Dataset and Large Language Model for Finance',
 'FLAb: Benchmarking deep learning methods for antibody fitness pr

In [75]:
reviews_nips_2023 ={}
for i in range(len(nips_2023_titles)):
    reviews_nips_2023[nips_2023_titles[i]] = get_reviews_fast(nips_2023_titles[i], "NeurIPS", 2023, reply_type="Official_Review")

reviews_nips_2023
    

Getting V2 Notes: 100%|█████████▉| 3391/3395 [01:10<00:00, 48.11it/s]


{'Biologically-inspired adaptive learning in the Hopfield-network based self-optimization model': {},
 'Flexible Attention-Based Multi-Policy Fusion for Efficient Deep Reinforcement Learning': {'review_20': {'summary': 'This paper defines the Knowledge-Grounded RL setting, a general RL setting for integrating knowledge (in the form of policies) into a policy to learn new tasks efficiently. Essentially this setting is similar to the MDP setting except that the agent is also given a set of knowledge policies to utilize. The paper also introduces a system/architecture within this KGRL setting, called Knowledge-Inclusive Attention Network (KIAN). The aim is to improve RL that is grounded on external knowledge policies. The paper outlines five desirable human-like properties they desire in their agents: knowledge-acquirable, sample-efficient, generalizable, compositional, and incremental. Moreover, they formally define these so that they are measurable within the KGRL setting (e.g., for eva

In [76]:
reviews_nips_2023.to_parquet("nips_2023_reviews.parquet")

AttributeError: 'dict' object has no attribute 'to_parquet'

In [77]:
import json


with open("nips_2023_reviews.json", "w", encoding="utf-8") as f:
    json.dump(reviews_nips_2023, f, indent=2)

In [ ]:
def norm(s):
    return str(s).strip().lower()

def get_title(sub):
    t = sub.content.get("title")
    return t.get("value") if isinstance(t, dict) else t

def unwrap(v):
    return v.get("value") if isinstance(v, dict) else v

def fetch_reviews_by_titles(conf, year, titles, reply_type="Official_Review"):
    """
    conf: e.g. "NeurIPS" or "ICLR"
    year: int, e.g. 2023
    titles: list of paper titles (strings)
    reply_type: "Official_Review" / "Meta_Review" / etc.

    Returns: dict {original_title: [review_dicts]}
    """
    venue_id = f"{conf}.cc/{year}/Conference"
    submissions = client_v2.get_all_notes(
        invitation=f"{venue_id}/-/{submission_name}",
        details="replies"
    )

    # Build index: normalized title -> list of reviews
    index = {}
    for submission in submissions:
        title_norm = norm(get_title(submission))
        reviews = []
        for reply in submission.details.get("replies", []):
            if any(inv.endswith(reply_type) for inv in reply.get("invitations", [])):
                content = reply.get("content", {})
                reviews.append({
                    "summary": unwrap(content.get("summary")),
                    "strengths": unwrap(content.get("strengths")),
                    "weaknesses": unwrap(content.get("weaknesses")),
                })
        index[title_norm] = reviews

    # Map back to the original titles given
    return {t: index.get(norm(t), []) for t in titles}


In [79]:
nips_2022 = []
for i in range(len(df)):
    if df.iloc[i]['conference'] == 'NeurIPS' and df.iloc[i]['year'] == 2022:
        nips_2022.append(df.iloc[i]['title'])

In [132]:
iclr_2023 = []
for i in range(len(df)):
    if df.iloc[i]['conference'] == 'ICLR' and df.iloc[i]['year'] == 2023:
        iclr_2023.append(df.iloc[i]['title'])

In [175]:
iclr_2023

['UniMax: Fairer and More Effective Language Sampling for Large-Scale Multilingual Pretraining',
 'StyleMorph: Disentangled 3D-Aware Image Synthesis with a 3D Morphable StyleGAN',
 'Revisiting Robustness in Graph Machine Learning']

In [133]:
reviews_iclr_2023 = fetch_reviews_by_titles("ICLR", 2023, iclr_2023, reply_type="Official_Review")

In [134]:
reviews_iclr_2023

{'UniMax: Fairer and More Effective Language Sampling for Large-Scale Multilingual Pretraining': [],
 'StyleMorph: Disentangled 3D-Aware Image Synthesis with a 3D Morphable StyleGAN': [],
 'Revisiting Robustness in Graph Machine Learning': []}

In [131]:
import json


with open("iclr_2024_reviews.json", "w", encoding="utf-8") as f:
    json.dump(reviews_iclr_2024, f, indent=2)

In [88]:
reviews_nips_2023_meta

{'Biologically-inspired adaptive learning in the Hopfield-network based self-optimization model': [],
 'Flexible Attention-Based Multi-Policy Fusion for Efficient Deep Reinforcement Learning': [],
 'Conditional Matrix Flows for Gaussian Graphical Models': [],
 'Matrix Compression via Randomized Low Rank and Low Precision Factorization': [],
 'Color Equivariant Convolutional Networks': [],
 'VideoComposer: Compositional Video Synthesis with Motion Controllability': [],
 'SiT Dataset: Socially Interactive Pedestrian Trajectory Dataset for Social Navigation Robots': [],
 'Automatic Clipping: Differentially Private Deep Learning Made Easier and Stronger': [],
 'A Theoretical Analysis of the Test Error of Finite-Rank Kernel Ridge Regression': [],
 'Comparing Optimization Targets for Contrast-Consistent Search': [],
 'Fine-Tuning Language Models with Just Forward Passes': [],
 'PIXIU: A Comprehensive Benchmark, Instruction Dataset and Large Language Model for Finance': [],
 'FLAb: Benchmarki

In [97]:
nips_2022

['The Hessian Screening Rule',
 'CW-ERM: Improving Autonomous Driving Planning with Closed-loop Weighted Empirical Risk Minimization',
 'Identifying the Context Shift between Test Benchmarks and Production Data',
 'Neural Unbalanced Optimal Transport via Cycle-Consistent Semi-Couplings',
 'Privacy-Preserving Machine Learning for Collaborative Data Sharing via Auto-encoder Latent Space Embeddings',
 'Differentially Private CutMix for Split Learning with Vision Transformer',
 'DeepJoint: Robust Survival Modelling Under Clinical Presence Shift',
 'Fast kinematics modeling for conjunction with lens image modeling',
 'PatchRot: A Self-Supervised Technique for Training Vision Transformers',
 'DARTFormer: Finding The Best Type Of Attention',
 'Langevin Autoencoders for Learning Deep Latent Variable Models',
 'Self-supervised detection of atmospheric phenomena from remotely sensed synthetic aperture radar imagery',
 'Deconvolving Detector Effects for Distribution Moments',
 'Conformal Semantic

In [112]:
venue_id = "NeurIPS.cc/2022/Conference"

# list all invitations under this venue
invs = client_v1.get_all_invitations(venue_id)

# find submission invitations
invs

[]

In [101]:
submissions = client.get_all_notes(
    invitation="NeurIPS.cc/2022/Conference/-/Submission"
    )

In [102]:
submissions

[]

In [111]:
import openreview

client = openreview.Client(baseurl="https://api.openreview.net")

venue_prefix = "NeurIPS.cc/2022/Conference"

# 1) Find all review invitations under this venue
invs = client.get_invitations(client, id = venue_prefix)
review_invs = [inv.id for inv in invs if inv.id.endswith("Official_Review")]

print("Found review invitations:", review_invs[:5])

# 2) Pull all reviews from those invitations
all_reviews = []
for inv_id in review_invs:
    all_reviews += list(openreview.tools.iterate_notes(client, invitation=inv_id))

print("Total reviews:", len(all_reviews))


TypeError: Client.get_invitations() got multiple values for argument 'id'

In [109]:
import openreview
from openreview import tools

client = openreview.Client(baseurl="https://api.openreview.net")
venue_prefix = "NeurIPS.cc/2022/Conference"

invs = list(tools.iterate_invitations(client, prefix=venue_prefix))
review_invs = [inv.id for inv in invs if inv.id.endswith("Official_Review")]
review_invs[:5]


AttributeError: module 'openreview.tools' has no attribute 'iterate_invitations'

In [119]:
# Double-blind venues

submissions = client_v1.get_all_notes(invitation = 'NeurIPS.cc/2022/Conference/-/Blind_Submission', details='directReplies,original')
blind_notes = {note.id: note for note in submissions}
all_decision_notes = [] 
for submission_id, submission in blind_notes.items(): 
        all_decision_notes = all_decision_notes + [reply for reply in submission.details["directReplies"] if reply["invitation"].endswith("Decision")]

accepted_submissions = []

for decision_note in all_decision_notes:
    if 'Accept' in decision_note["content"]['decision']:
        accepted_submissions.append(blind_notes[decision_note['forum']].details['original'])

Getting V1 Notes: 100%|█████████▉| 2821/2824 [00:40<00:00, 69.54it/s]


KeyError: 'original'

In [126]:
submissions = client_v1.get_all_notes(
    invitation="NeurIPS.cc/2022/Conference/-/Blind_Submission", details='directReplies'
    )

Getting V1 Notes: 100%|█████████▉| 2821/2824 [00:39<00:00, 70.69it/s]


In [138]:
titles_2022 = [(sub.content.get('title')) for sub in submissions]

In [140]:
titles_2022.index("The Hessian Screening Rule")

1977

In [171]:
def fetch_reviews_v1(titles, conf, year):
    venue_id = f"{conf}.cc/{year}/Conference"
    submissions = client_v1.get_all_notes(invitation=f"{venue_id}/-/Blind_Submission", details="directReplies")
    indices = []
    indices_without_reviews = []
    all_titles = [(sub.content.get('title')) for sub in submissions]
    for title in titles:
        if title in all_titles:
            indices.append(all_titles.index(title))
        else:
            indices_without_reviews.append(title)
    title_to_reviews = {}
    for idx in indices:
        review_idx_lst = submissions[idx].details.get("directReplies")
        reviews = {}
        for i in range(len(review_idx_lst)):
            summary = review_idx_lst[i].get('content').get('summary') if review_idx_lst[i].get('content').get('summary') else None
            strengths_and_weaknesses = review_idx_lst[i].get('content').get('strengths_and_weaknesses') if review_idx_lst[i].get('content').get('strengths_and_weaknesses') else None
            reviews[f"review{i+1}"] = {"summary": summary, "strengths_and_weaknesses": strengths_and_weaknesses}
        title = submissions[idx].content.get('title')
        title_to_reviews[title] = reviews
    for title in indices_without_reviews:
        title_to_reviews[title] = {}
    return title_to_reviews

In [176]:
reviews_iclr_2023 = fetch_reviews_v1(iclr_2023, "ICLR", 2023)

Getting V1 Notes: 100%|█████████▉| 3788/3792 [00:27<00:00, 136.18it/s]


In [177]:
reviews_iclr_2023


{'UniMax: Fairer and More Effective Language Sampling for Large-Scale Multilingual Pretraining': {'review1': {'summary': None,
   'strengths_and_weaknesses': None},
  'review2': {'summary': None, 'strengths_and_weaknesses': None},
  'review3': {'summary': None, 'strengths_and_weaknesses': None},
  'review4': {'summary': None, 'strengths_and_weaknesses': None},
  'review5': {'summary': None, 'strengths_and_weaknesses': None},
  'review6': {'summary': None, 'strengths_and_weaknesses': None}},
 'StyleMorph: Disentangled 3D-Aware Image Synthesis with a 3D Morphable StyleGAN': {'review1': {'summary': None,
   'strengths_and_weaknesses': None},
  'review2': {'summary': None, 'strengths_and_weaknesses': None},
  'review3': {'summary': None, 'strengths_and_weaknesses': None},
  'review4': {'summary': None, 'strengths_and_weaknesses': None},
  'review5': {'summary': None, 'strengths_and_weaknesses': None},
  'review6': {'summary': None, 'strengths_and_weaknesses': None},
  'review7': {'summary'

In [173]:
reviews_2022

{'The Hessian Screening Rule': {'review1': {'summary': 'The paper proposes a screening strategy for L1 sparse modelings. The basic idea is to use a prediction of an optimal solution as a function of the regularization parameter, derived from optimal conditions (which is aka solution path). The authors further propose combining working set selection screening by predicted solution with so-called strong rule to make screening more efficient. ',
   'strengths_and_weaknesses': "Overall, the paper is easy to follow and technical quality is fine. The purpose is clear and the procedure is written in detail. However, a critical issue is that the idea of main proposal (Hessian screening) is not novel. Approaches based on a similar idea have been studies in the context of the path following though it is not fully mentioned. Detailed comments are as follows.\n\nClosely related papers in the path following literature are missed. For example, since the following two papers contains conceptually qui

In [174]:
import json


with open("nips_2022_reviews.json", "w", encoding="utf-8") as f:
    json.dump(reviews_2022, f, indent=2)

In [162]:
submissions[1977].details['directReplies'][2].get('content').get('summary')

'A Hessian screening rule for lasso and its generalized linear model extension for logistic regression was presented to take advantage of the high-order information for more efficient screening, specifically in cases with highly correlated predictors/covariates. The proposed Hessian screening rule together with several speedup tricks has been shown to be effective in both simulated and real datasets. '

In [160]:
submissions[1977].content.get('title')

'The Hessian Screening Rule'

In [158]:
def unwrap(v):
    return v.get("value") if isinstance(v, dict) else v

def get_reviews_v1(conf, year, titles):
    venue_id = f"{conf}.cc/{year}/Conference/-/Blind_Submission"
    submissions = client_v1.get_all_notes(invitation=venue_id, details="directReplies")

    wanted = set(titles)
    title_to_reviews = {}

    for sub in submissions:
        title = unwrap(sub.content.get("title"))
        if title not in wanted:
            continue

        reviews = {}
        i = 1
        for reply in sub.details.get("directReplies", []):
            if any(inv.endswith("Official_Review") for inv in reply.get("invitations", [])):
                content = reply.get("content")  # same as your working example
                reviews[i] = content
                i += 1

        title_to_reviews[title] = reviews

    return title_to_reviews
reviews_2022 = get_reviews_v1("NeurIPS", 2022, nips_2022)

Getting V1 Notes: 100%|█████████▉| 2821/2824 [00:22<00:00, 126.38it/s]


In [153]:
def unwrap(v):
    return v.get("value") if isinstance(v, dict) else v

def get_reviews_v1_once(conf, year, titles):
    # One API call (with pagination handled internally)
    invitation = f"{conf}.cc/{year}/Conference/-/Blind_Submission"
    submissions = client_v1.get_all_notes(invitation=invitation, details="directReplies")

    wanted = set(titles)
    title_to_reviews = {}

    for sub in submissions:
        title = unwrap(sub.content.get("title"))
        if title not in wanted:
            continue

        reviews = {}
        i = 1
        for reply in sub.details.get("directReplies", []):
            if any(inv.endswith("Official_Review") for inv in reply.get("invitations", [])):
                content = reply.get("content", {})
                reviews[f"review{i}"] = {
                    "summary": unwrap(content.get("summary")),
                    "strengths_and_weaknesses": unwrap(content.get("strengths_and_weaknesses")),
                }
                i += 1

        title_to_reviews[title] = reviews

    return title_to_reviews
reviews_2022 = get_reviews_v1_once("NeurIPS", 2022, nips_2022)

Getting V1 Notes: 100%|█████████▉| 2821/2824 [00:33<00:00, 83.77it/s]


In [159]:
reviews_2022

{'Langevin Autoencoders for Learning Deep Latent Variable Models': {},
 'The Hessian Screening Rule': {},
 'Uni[MASK]: Unified Inference in Sequential Decision Problems': {}}

In [179]:
df[df["conference"] == "ICML"].iloc[0]['title']

'Transformers, parallel computation, and logarithmic depth'